In [2]:
#do not use pip install -- put packages that need to be installed inside environment to 'requirements.txt'

""" REQUIRED PACKAGES HERE:"""
import numpy as np
import pandas as pd
import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords
#from textblob import TextBlob
#from textblob import Word
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


##temp packages
#necessary packages for querying data:
import sys
#import lyricsgenius as genius
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import re
import json


#plotting 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')


### Example of path to .json file: "/home/jovyan/work/mpd.slice.29000-29999.json"

## Claire's Methods:

In [3]:
def convert_json_to_df(json_file_path):
    """
    Converts the json file of the Spotify Million Playlist Dataset to a dataframe. 
    Returns the playlist field and excludes the info field.
    
    input: 
    json_file (string) - json file path
    
    output:
    dataframe of the playlist
    """
    
    with open(json_file_path) as json_file:
        df0 = json.load(json_file)
    
    df = pd.DataFrame.from_dict(df0['playlists'])
    
    return df

In [4]:
def extract_tracks(df):
    """
    Extracts the tracks from the in the df, which is a dictionary, and converts it to a dataframe
    
    input: 
    df - dataframe of the playlist
    
    output:
    dataframe of all the songs in the playlist
    
    """
    # initializing a bigger dataframe
    new_df = pd.DataFrame()
    
    for i in range(len(df['tracks'])):
        
        # converting tracks dictionary to dataframe
        data_playlist = pd.DataFrame.from_dict(df['tracks'][i])

        # getting the 'pid' from the original df to keep track of what playlist the song was in
        data_playlist['pid'] = df['pid'][i]

        # adding the the songs from the playlist to the bigger dataset
        new_df = new_df.append(data_playlist)
    
    return new_df.reset_index(drop=True)

## Ferdie: Adding onto Claire's Methods:

In [5]:
tdf = convert_json_to_df("/home/jovyan/work/mpd.slice.29000-29999.json")
tdf_a = extract_tracks(tdf)
len(tdf_a)

67684

In [6]:
tdf_a.head()

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,pid
0,0,Bobby Darin,spotify:track:3E5ndyOfO6vFDEIE42HA8o,spotify:artist:0EodhzA6yW1bIdD5B4tcmJ,Mack The Knife,spotify:album:5MsJK0kqiYIJDmd3cjkGMn,184333,That's All,29000
1,1,Frank Sinatra,spotify:track:2y8Eez5cFFf2JzD546LThM,spotify:artist:1Mxqyy3pSjf8kZZL4QVxS0,Fly Me To The Moon,spotify:album:6No1Xr0VqSg5SjFAbyRsZQ,147000,Sinatra/Basie: The Complete Reprise Studio Rec...,29000
2,2,Etta James,spotify:track:0CmIALzGn4vHIHJG4n3Q4z,spotify:artist:0iOVhN3tnSvgDbcg25JoJb,At Last - Single Version,spotify:album:2pBhXw3Hi1hBf8FpAtE101,182400,At Last!,29000
3,3,Ella Fitzgerald,spotify:track:3Bbbz0IGORWZSLf9UqsAL4,spotify:artist:5V0MlUE1Bft0mbLlND7FJz,Dream A Little Dream Of Me,spotify:album:6yKeev1MnyUbFjhNHWsFFK,182266,C'est Si Bon,29000
4,4,The Chordettes,spotify:track:433nBELvOOtHlZFUO8mZml,spotify:artist:62GnBjssWSXBlKvQohT2Bk,Mister sandman,spotify:album:6jGd5RtEt0g4X4xvoPk1XC,147266,Greatest Hits,29000


In [7]:
tdf_final = tdf_a.drop_duplicates(['track_uri'])
len(tdf_final)

36075

In [8]:
tdf_final.head()

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,pid
0,0,Bobby Darin,spotify:track:3E5ndyOfO6vFDEIE42HA8o,spotify:artist:0EodhzA6yW1bIdD5B4tcmJ,Mack The Knife,spotify:album:5MsJK0kqiYIJDmd3cjkGMn,184333,That's All,29000
1,1,Frank Sinatra,spotify:track:2y8Eez5cFFf2JzD546LThM,spotify:artist:1Mxqyy3pSjf8kZZL4QVxS0,Fly Me To The Moon,spotify:album:6No1Xr0VqSg5SjFAbyRsZQ,147000,Sinatra/Basie: The Complete Reprise Studio Rec...,29000
2,2,Etta James,spotify:track:0CmIALzGn4vHIHJG4n3Q4z,spotify:artist:0iOVhN3tnSvgDbcg25JoJb,At Last - Single Version,spotify:album:2pBhXw3Hi1hBf8FpAtE101,182400,At Last!,29000
3,3,Ella Fitzgerald,spotify:track:3Bbbz0IGORWZSLf9UqsAL4,spotify:artist:5V0MlUE1Bft0mbLlND7FJz,Dream A Little Dream Of Me,spotify:album:6yKeev1MnyUbFjhNHWsFFK,182266,C'est Si Bon,29000
4,4,The Chordettes,spotify:track:433nBELvOOtHlZFUO8mZml,spotify:artist:62GnBjssWSXBlKvQohT2Bk,Mister sandman,spotify:album:6jGd5RtEt0g4X4xvoPk1XC,147266,Greatest Hits,29000


In [9]:
track_uri_tuple = [(x,y,z) for x,y,z in zip(tdf_final['artist_name'], tdf_final['track_name'], tdf_final['track_uri'])]
track_uri_tuple

[('Bobby Darin', 'Mack The Knife', 'spotify:track:3E5ndyOfO6vFDEIE42HA8o'),
 ('Frank Sinatra',
  'Fly Me To The Moon',
  'spotify:track:2y8Eez5cFFf2JzD546LThM'),
 ('Etta James',
  'At Last - Single Version',
  'spotify:track:0CmIALzGn4vHIHJG4n3Q4z'),
 ('Ella Fitzgerald',
  'Dream A Little Dream Of Me',
  'spotify:track:3Bbbz0IGORWZSLf9UqsAL4'),
 ('The Chordettes', 'Mister sandman', 'spotify:track:433nBELvOOtHlZFUO8mZml'),
 ('Sam Cooke',
  '(What A) Wonderful World - Remastered',
  'spotify:track:27K3ZDS5B4fwjhwyihrdzC'),
 ('The Temptations', 'My Girl', 'spotify:track:6RrXd9Hph4hYR4bf3dbM6H'),
 ('Louis Armstrong',
  'La Vie En Rose - Single Version',
  'spotify:track:0AX6pLXtyR2vPLv2KYErAg'),
 ('Frank Sinatra', "Somethin' Stupid", 'spotify:track:3cVWNLd0uEHkc2tnBoE3Ay'),
 ('Nina Simone', 'Feeling Good', 'spotify:track:61S0mEfdRtJ62Fc7fPqx0U'),
 ('Louis Armstrong',
  'What A Wonderful World - Single Version',
  'spotify:track:29U7stRjqHU6rMiS8BfaI9'),
 ('Marvin Gaye',
  "Ain't No Mountai

In [10]:
def get_track_uri_tuples(df): 
    """
    input:
    df- dataframe of playlist from 'extract' tracks

    output- removes duplicates of tracks and then returns all track uris of that df
    
    """
    df_final = df.drop_duplicates(['track_uri'])
    track_uri_tuple = [(x,y,z) for x,y,z in zip(df_final['artist_name'], df_final['track_name'], df_final['track_uri'])]
    return track_uri_tuple

In [11]:
def get_track_uri(track_uri_tuple):
    """
    input:
    track_uri_tuple - list of track_uris_tuples (artist name, song, track_uris)

    output- just track_uris in same order for spotipy
    
    """

    track_uris = [x[2] for x in track_uri_tuple]
    return track_uris


track_uris = get_track_uri(track_uri_tuple)

## Ferdie: Extracting Spotify Features

` first need to authenticate properly, then given a list of track uri and name, gather attributes for that feature`

In [12]:
client_credentials_manager = SpotifyClientCredentials(client_id='911b425bc8c3496297b0a66d3df0b236', client_secret='5f581823761649768f3ca1d3a7cfc6de')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [13]:
##EXPLORING WHAT YOU CAN GET THROUGH TRACK URI:
"""
1. og spotify audio features
2. 'available_markets'
3. 'release_date'
4. 'popularity'
5. 'is_explicit'
"""
sp.tracks(tracks=track_uris[0:50])['tracks'][0]

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0EodhzA6yW1bIdD5B4tcmJ'},
    'href': 'https://api.spotify.com/v1/artists/0EodhzA6yW1bIdD5B4tcmJ',
    'id': '0EodhzA6yW1bIdD5B4tcmJ',
    'name': 'Bobby Darin',
    'type': 'artist',
    'uri': 'spotify:artist:0EodhzA6yW1bIdD5B4tcmJ'}],
  'available_markets': ['AD',
   'AE',
   'AL',
   'AR',
   'AT',
   'AU',
   'BA',
   'BE',
   'BG',
   'BH',
   'BO',
   'BR',
   'BY',
   'CA',
   'CH',
   'CL',
   'CO',
   'CR',
   'CY',
   'CZ',
   'DE',
   'DK',
   'DO',
   'DZ',
   'EC',
   'EE',
   'EG',
   'ES',
   'FI',
   'FR',
   'GB',
   'GR',
   'GT',
   'HK',
   'HN',
   'HR',
   'HU',
   'ID',
   'IE',
   'IL',
   'IN',
   'IS',
   'IT',
   'JO',
   'JP',
   'KW',
   'KZ',
   'LB',
   'LI',
   'LT',
   'LU',
   'LV',
   'MA',
   'MC',
   'MD',
   'ME',
   'MK',
   'MT',
   'MX',
   'MY',
   'NI',
   'NL',
   'NO',
   'NZ',
   'OM',
   'PA',
   'PE',
   'PH',
   'PL',
   'PS',


In [15]:
def get_audio_features(track_uri_tuple):
    """
    input: track_uri_tuple
    output: a dataframe with all audio_features 

    """
    track_uris = get_track_uri(track_uri_tuple)
    audio_features_dict = {}
    track_uri_length = len(track_uri_tuple)

    ## UNFORTUNATELY YOU CAN ONLY GRAB 100 AUDIO FEATURES AT A TIME, HAVE TO LOOP AND MANUALLY ADD TO DICTIONARY.
    i = 0
    while i < track_uri_length:
        af = sp.audio_features(tracks=track_uris[0+i:100+i])
        #indexing might be off near the end, need if statement if there's remainder, get rest of track >> probably use try and except

        for b in np.arange(len(af)):
            current_tuple = track_uri_tuple[i+b]
            audio_features_dict[current_tuple] = af[b]
        i+= 100
    
    xf = pd.DataFrame.from_dict(audio_features_dict, 'index')
    df_final = xf.reset_index().rename(columns={"level_0": "Artist Name", "level_1": "Song Name"}).drop('level_2', axis =1)

    return df_final

get_audio_features(track_uri_tuple)

,Artist Name,Song Name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Bobby Darin,Mack The Knife,0.549,0.529,3,-12.291,0,0.1080,0.76000,0.00000,0.2060,0.464,82.755,audio_features,3E5ndyOfO6vFDEIE42HA8o,spotify:track:3E5ndyOfO6vFDEIE42HA8o,https://api.spotify.com/v1/tracks/3E5ndyOfO6vF...,https://api.spotify.com/v1/audio-analysis/3E5n...,184333,4
1,Frank Sinatra,Fly Me To The Moon,0.657,0.260,7,-14.251,1,0.0521,0.47400,0.00000,0.0656,0.378,119.358,audio_features,2y8Eez5cFFf2JzD546LThM,spotify:track:2y8Eez5cFFf2JzD546LThM,https://api.spotify.com/v1/tracks/2y8Eez5cFFf2...,https://api.spotify.com/v1/audio-analysis/2y8E...,147000,4
2,Etta James,At Last - Single Version,0.168,0.326,5,-9.981,1,0.0324,0.72100,0.00060,0.3160,0.322,174.794,audio_features,0CmIALzGn4vHIHJG4n3Q4z,spotify:track:0CmIALzGn4vHIHJG4n3Q4z,https://api.spotify.com/v1/tracks/0CmIALzGn4vH...,https://api.spotify.com/v1/audio-analysis/0CmI...,182400,3
3,Ella Fitzgerald,Dream A Little Dream Of Me,0.631,0.127,0,-15.333,1,0.0949,0.93900,0.00000,0.0732,0.322,76.559,audio_features,3Bbbz0IGORWZSLf9UqsAL4,spotify:track:3Bbbz0IGORWZSLf9UqsAL4,https://api.spotify.com/v1/tracks/3Bbbz0IGORWZ...,https://api.spotify.com/v1/audio-analysis/3Bbb...,182267,4
4,The Chordettes,Mister sandman,0.616,0.298,9,-13.191,1,0.0520,0.84100,0.00000,0.0605,0.569,108.779,audio_features,433nBELvOOtHlZFUO8mZml,spotify:track:433nBELvOOtHlZFUO8mZml,https://api.spotify.com/v1/tracks/433nBELvOOtH...,https://api.spotify.com/v1/audio-analysis/433n...,147267,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36070,6LACK,Rules,0.682,0.758,8,-6.130,0,0.1550,0.17800,0.00158,0.0964,0.638,147.045,audio_features,6HmWy7ttkeZGG2KEtt8AqX,spotify:track:6HmWy7ttkeZGG2KEtt8AqX,https://api.spotify.com/v1/tracks/6HmWy7ttkeZG...,https://api.spotify.com/v1/audio-analysis/6HmW...,211867,4
36071,Lil Uzi Vert,How To Talk,0.728,0.730,5,-5.402,1,0.2130,0.74100,0.00000,0.7050,0.736,136.052,audio_features,0ro5QCkSCtbfhMHnvMase3,spotify:track:0ro5QCkSCtbfhMHnvMase3,https://api.spotify.com/v1/tracks/0ro5QCkSCtbf...,https://api.spotify.com/v1/audio-analysis/0ro5...,201544,4
36072,blackbear,90210 (feat. G-Eazy),0.707,0.707,11,-6.161,0,0.0375,0.03060,0.00000,0.1270,0.231,104.039,audio_features,4hdog9vyyqG9pcppG2Izek,spotify:track:4hdog9vyyqG9pcppG2Izek,https://api.spotify.com/v1/tracks/4hdog9vyyqG9...,https://api.spotify.com/v1/audio-analysis/4hdo...,223296,4
36073,G-Eazy,Eyes Closed,0.715,0.763,11,-6.329,0,0.2400,0.04680,0.00000,0.1030,0.395,133.987,audio_features,0rZ3xYyjHM9B1IBD9ixZ08,spotify:track:0rZ3xYyjHM9B1IBD9ixZ08,https://api.spotify.com/v1/tracks/0rZ3xYyjHM9B...,https://api.spotify.com/v1/audio-analysis/0rZ3...,204054,4


## Ferdie: Looping across all .json files located in directory

In [16]:
def convert_json_to_features(path):
    """
    input: file name path (example: "/home/jovyan/work/mpd.slice.29000-29999.json")
    output: final_df with audio features
    """
    ##Initial json to df conversion, removes duplicates found in list
    df = convert_json_to_df(path)
    df_a = extract_tracks(df)
    df_final = df_a.drop_duplicates(['track_uri'])
    ## converting track uri to dataframe with features
    tuples = get_track_uri_tuples(df_final)
    final_df = get_audio_features(tuples)
    return final_df

In [17]:
import os, json

def convert_json_to_features_direct(directory):

    #directory #example = '/home/jovyan/work/'
    json_files = [pos_json for pos_json in os.listdir(directory) if pos_json.endswith('.json')]
    
    initial_path = directory + json_files[0]
    initial_df = convert_json_to_features(initial_path)

    for i in np.arange(1, len(json_files)):
        new_path = directory + json_files[i]
        new_df = convert_json_to_features(new_path)
        initial_df = pd.concat([initial_df, new_df])
    return initial_df

In [18]:
t1 = convert_json_to_features_direct('/home/jovyan/work/')

In [19]:
t1.shape

(139915, 20)

In [20]:
t1_final = t1.drop_duplicates(['uri']).reset_index(drop = True) #Final DF after 4 json files (a lot of duplicates)

In [21]:
### CellOutputSizeError --> commented out for now
#t1_final.to_csv()

# Psalm: KNN Exploration

Inspired by: https://www.kaggle.com/danielgrijalvas/finding-songs-that-match-sw-s-style-using-knn

To Look Into: 
- Using k-means instead of k nearest neighbors given that our labels aren't known.
    - Inspiration: https://towardsdatascience.com/clustering-music-to-create-your-personal-playlists-on-spotify-using-python-and-k-means-a39c4158589a
    - https://medium.com/datadriveninvestor/k-means-clustering-using-spotify-data-45435a7cd32a
    - Based on clustering by similarity
    - Once clustered, randomize song selection to build playlist?
- Number of song inputs

Potential KNN Process:
- (optional) for each song, determine KBest defining features
- EASY/RISKY: Use 'pid' to find similar songs? --> knn classifies song and associates it with a playlist, go through playlist to find similar songs?
- HARD/RISKY: Use defining features to determine class value (boolean) for whether song in dataset matches the input
    - values within a certain interval --> class = 1; else, class = 0
    - Preprocessing: Create columns in df to reflect whether intervals or passed

- Fit to KNN. 
- Predict with test set.
- Use knn.predict_proba(*test set*) to determine simlarity

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.BallTree.html
Fit BallTree, apply KNN? --> Method used by data x team

In [22]:
def drop_string(df):
    """
    Drops the non-integer and non-float type columns for fitting to KNN model.
    Returns the formatted DataFrame.
    
    input: 
    df (DataFrame) - 
    
    output:
    Formatted dataframe of the playlist
    """
    return df.select_dtypes(exclude=['string', 'object'])

In [23]:
t1_final.head()

,Artist Name,Song Name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Bobby Darin,Mack The Knife,0.549,0.529,3,-12.291,0,0.1080,0.760,0.0000,0.2060,0.464,82.755,audio_features,3E5ndyOfO6vFDEIE42HA8o,spotify:track:3E5ndyOfO6vFDEIE42HA8o,https://api.spotify.com/v1/tracks/3E5ndyOfO6vF...,https://api.spotify.com/v1/audio-analysis/3E5n...,184333,4
1,Frank Sinatra,Fly Me To The Moon,0.657,0.260,7,-14.251,1,0.0521,0.474,0.0000,0.0656,0.378,119.358,audio_features,2y8Eez5cFFf2JzD546LThM,spotify:track:2y8Eez5cFFf2JzD546LThM,https://api.spotify.com/v1/tracks/2y8Eez5cFFf2...,https://api.spotify.com/v1/audio-analysis/2y8E...,147000,4
2,Etta James,At Last - Single Version,0.168,0.326,5,-9.981,1,0.0324,0.721,0.0006,0.3160,0.322,174.794,audio_features,0CmIALzGn4vHIHJG4n3Q4z,spotify:track:0CmIALzGn4vHIHJG4n3Q4z,https://api.spotify.com/v1/tracks/0CmIALzGn4vH...,https://api.spotify.com/v1/audio-analysis/0CmI...,182400,3
3,Ella Fitzgerald,Dream A Little Dream Of Me,0.631,0.127,0,-15.333,1,0.0949,0.939,0.0000,0.0732,0.322,76.559,audio_features,3Bbbz0IGORWZSLf9UqsAL4,spotify:track:3Bbbz0IGORWZSLf9UqsAL4,https://api.spotify.com/v1/tracks/3Bbbz0IGORWZ...,https://api.spotify.com/v1/audio-analysis/3Bbb...,182267,4
4,The Chordettes,Mister sandman,0.616,0.298,9,-13.191,1,0.0520,0.841,0.0000,0.0605,0.569,108.779,audio_features,433nBELvOOtHlZFUO8mZml,spotify:track:433nBELvOOtHlZFUO8mZml,https://api.spotify.com/v1/tracks/433nBELvOOtH...,https://api.spotify.com/v1/audio-analysis/433n...,147267,4


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Using 'mode' as a placeholder for now to test process. Switch out for class once decided upon.
y = t1_final['key']

X = t1_final.drop(columns = 'key')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)


In [25]:
from sklearn.neighbors import KNeighborsClassifier

formatted_X_train = drop_string(X_train)

knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(formatted_X_train, y_train)

formatted_X_test = drop_string(X_test)
preds = knn.predict(formatted_X_test)

In [26]:
# probabililty of correctness
probabilities = knn.predict_proba(formatted_X_test)
probabilities

array([[0. , 0.4, 0.2, ..., 0. , 0. , 0.2],
       [0. , 0. , 0. , ..., 0.2, 0. , 0. ],
       [0. , 0. , 0. , ..., 0.2, 0.2, 0.2],
       ...,
       [0.2, 0. , 0. , ..., 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, ..., 0. , 0. , 0. ],
       [0. , 0.4, 0.2, ..., 0.2, 0. , 0. ]])

In [27]:
X_test['Probability of Correctness'] = [prob[1] for prob in probabilities]
X_test.head()

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Artist Name,Song Name,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,Probability of Correctness
4879,Ed Tullett,Faux,0.242,0.421,-12.129,1,0.0309,0.860000,0.233,0.1080,0.250,144.937,audio_features,0d9UAfLZnNNKhr5zUd9nfv,spotify:track:0d9UAfLZnNNKhr5zUd9nfv,https://api.spotify.com/v1/tracks/0d9UAfLZnNNK...,https://api.spotify.com/v1/audio-analysis/0d9U...,272172,3,0.4
74842,Children Of Bodom,Are You Dead Yet,0.467,0.967,-2.990,1,0.0491,0.000006,0.310,0.3010,0.312,95.991,audio_features,7i6UQPAHgqgMGJJK7elUGS,spotify:track:7i6UQPAHgqgMGJJK7elUGS,https://api.spotify.com/v1/tracks/7i6UQPAHgqgM...,https://api.spotify.com/v1/audio-analysis/7i6U...,234880,4,0.0
31323,Amy Winehouse,"I Heard Love Is Blind - Live At Concorde, Brig...",0.708,0.403,-9.266,1,0.0767,0.240000,0.000,0.9720,0.418,73.847,audio_features,1xYOULOeZuBdO3dvDz2BwH,spotify:track:1xYOULOeZuBdO3dvDz2BwH,https://api.spotify.com/v1/tracks/1xYOULOeZuBd...,https://api.spotify.com/v1/audio-analysis/1xYO...,147467,4,0.0
17409,Glee Cast,Landslide (Glee Cast Version featuring Gwyneth...,0.637,0.416,-7.595,1,0.0269,0.434000,0.000,0.1390,0.588,145.922,audio_features,0DuRaBhrsPbIyQfakgQrlI,spotify:track:0DuRaBhrsPbIyQfakgQrlI,https://api.spotify.com/v1/tracks/0DuRaBhrsPbI...,https://api.spotify.com/v1/audio-analysis/0DuR...,226760,4,0.0
80412,Keith Anderson,Every Time I Hear Your Name,0.562,0.757,-3.945,1,0.0253,0.277000,0.000,0.0745,0.299,93.020,audio_features,3aVCHTY93NQQbOcDDbwwXA,spotify:track:3aVCHTY93NQQbOcDDbwwXA,https://api.spotify.com/v1/tracks/3aVCHTY93NQQ...,https://api.spotify.com/v1/audio-analysis/3aVC...,263027,4,0.0


In [28]:
# Sorting by similarity
X_test.sort_values('Probability of Correctness', ascending=False)

,Artist Name,Song Name,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,Probability of Correctness
74300,Trina,B***h I Don't Need U,0.824,0.6630,-5.228,1,0.3220,0.04000,0.000000,0.0875,0.764,155.430,audio_features,5mzG3pceHwGZN1o6s3zAlq,spotify:track:5mzG3pceHwGZN1o6s3zAlq,https://api.spotify.com/v1/tracks/5mzG3pceHwGZ...,https://api.spotify.com/v1/audio-analysis/5mzG...,138173,4,0.8
42495,Regulo Caro,Cicatriiices,0.756,0.6880,-2.850,1,0.0299,0.62600,0.000000,0.1180,0.962,103.980,audio_features,795iKycRJtgee21MTf9nat,spotify:track:795iKycRJtgee21MTf9nat,https://api.spotify.com/v1/tracks/795iKycRJtge...,https://api.spotify.com/v1/audio-analysis/795i...,166173,4,0.8
85233,Foxx,Smokin Loud,0.818,0.5060,-10.240,1,0.3900,0.02410,0.000000,0.1080,0.704,168.954,audio_features,32pehhGftaYKKu80kjjrGr,spotify:track:32pehhGftaYKKu80kjjrGr,https://api.spotify.com/v1/tracks/32pehhGftaYK...,https://api.spotify.com/v1/audio-analysis/32pe...,315627,4,0.8
49957,George Strait,Heartland - Pure Country/Soundtrack Version,0.449,0.8900,-9.500,1,0.0500,0.05210,0.000077,0.1120,0.695,156.983,audio_features,0U4R6UqrJo43TmImPku9H8,spotify:track:0U4R6UqrJo43TmImPku9H8,https://api.spotify.com/v1/tracks/0U4R6UqrJo43...,https://api.spotify.com/v1/audio-analysis/0U4R...,138160,4,0.8
60327,Janet Jackson,Thinkin' Bout My Ex,0.584,0.5630,-8.071,1,0.0905,0.30100,0.000013,0.6480,0.150,78.989,audio_features,4HsitPIASpAezYrZal7xH9,spotify:track:4HsitPIASpAezYrZal7xH9,https://api.spotify.com/v1/tracks/4HsitPIASpAe...,https://api.spotify.com/v1/audio-analysis/4Hsi...,273773,4,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13086,Golden,You're Always Loved,0.578,0.3250,-7.078,1,0.0264,0.79000,0.000000,0.2980,0.555,157.833,audio_features,6YelAEU4fYMKXFxG5q235W,spotify:track:6YelAEU4fYMKXFxG5q235W,https://api.spotify.com/v1/tracks/6YelAEU4fYMK...,https://api.spotify.com/v1/audio-analysis/6Yel...,249873,4,0.0
45798,Chris Brown,Body Shots,0.598,0.5900,-7.593,0,0.1070,0.00604,0.000000,0.0425,0.515,144.781,audio_features,5bAwc4oM3fwqdidTHwaVnZ,spotify:track:5bAwc4oM3fwqdidTHwaVnZ,https://api.spotify.com/v1/tracks/5bAwc4oM3fwq...,https://api.spotify.com/v1/audio-analysis/5bAw...,222387,4,0.0
69072,Ahmad Lewis,Back In The Day - Remix,0.630,0.6410,-6.670,0,0.3690,0.34200,0.000000,0.5880,0.541,79.523,audio_features,5gUVdjfPWoaKoRiEMHBX6P,spotify:track:5gUVdjfPWoaKoRiEMHBX6P,https://api.spotify.com/v1/tracks/5gUVdjfPWoaK...,https://api.spotify.com/v1/audio-analysis/5gUV...,303373,4,0.0
52231,Fuel,Daniel,0.589,0.2870,-9.508,1,0.0316,0.80500,0.000006,0.0925,0.161,116.351,audio_features,0BGAynw6Ic9QhFwMihARQl,spotify:track:0BGAynw6Ic9QhFwMihARQl,https://api.spotify.com/v1/tracks/0BGAynw6Ic9Q...,https://api.spotify.com/v1/audio-analysis/0BGA...,269893,4,0.0


In [29]:
X_train.dtypes

Artist Name          object
Song Name            object
danceability        float64
energy              float64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
type                 object
id                   object
uri                  object
track_href           object
analysis_url         object
duration_ms           int64
time_signature        int64
dtype: object

## Exploring Spotify's Genres:

In [30]:
sp.recommendation_genre_seeds() #figuring out a way how this is attributed

{'genres': ['acoustic',
  'afrobeat',
  'alt-rock',
  'alternative',
  'ambient',
  'anime',
  'black-metal',
  'bluegrass',
  'blues',
  'bossanova',
  'brazil',
  'breakbeat',
  'british',
  'cantopop',
  'chicago-house',
  'children',
  'chill',
  'classical',
  'club',
  'comedy',
  'country',
  'dance',
  'dancehall',
  'death-metal',
  'deep-house',
  'detroit-techno',
  'disco',
  'disney',
  'drum-and-bass',
  'dub',
  'dubstep',
  'edm',
  'electro',
  'electronic',
  'emo',
  'folk',
  'forro',
  'french',
  'funk',
  'garage',
  'german',
  'gospel',
  'goth',
  'grindcore',
  'groove',
  'grunge',
  'guitar',
  'happy',
  'hard-rock',
  'hardcore',
  'hardstyle',
  'heavy-metal',
  'hip-hop',
  'holidays',
  'honky-tonk',
  'house',
  'idm',
  'indian',
  'indie',
  'indie-pop',
  'industrial',
  'iranian',
  'j-dance',
  'j-idol',
  'j-pop',
  'j-rock',
  'jazz',
  'k-pop',
  'kids',
  'latin',
  'latino',
  'malay',
  'mandopop',
  'metal',
  'metal-misc',
  'metalcore',


In [31]:
# Not sure what this is @Ferdie or Claire?
#sp.recommendations()

HTTP Error for GET to https://api.spotify.com/v1/recommendations returned 400 due to invalid request


SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/recommendations?limit=20:
 invalid request, reason: None

## K-Means Clustering (Psalm)

Finding the Best K for clustering and creating clusters

In [1]:
X = t1_final.copy() #hi i accidentally stopped the code from running so im just gonna restart running the notebook

In [33]:
sample_input = X.iloc[:1, :]
oh = drop_string(sample_input)
oh
X_train = X.iloc[1:, :]
#kmeans.predict(oh)
#kmeans.predict(drop_string(X).head().iloc[0, :].reshape(-1, 1))

### Finding the Best k for k clusters

In [ ]:
from sklearn.cluster import KMeans

### Finding best K's
max_diff = -999999
best_k = dict()

avg_sum_squares = []

# calculation method
for k in range(1, 20):
  kmeans = KMeans(n_clusters = k, random_state = 42)
  kmeans.fit(drop_string(X_train))

  avg_sum_squares += [kmeans.inertia_]

  if k == 1:
    prev_inertia = kmeans.inertia_

  else:
    slope = prev_inertia - kmeans.inertia_
    prev_inertia = kmeans.inertia_

    max_diff = max(max_diff, slope)

    if max_diff <= slope:
      best_k[max_diff] = k

plt.scatter(range(1, 20), avg_sum_squares)
print("Calculated Best K:", best_k[max_diff])

k = best_k[max_diff]


In [ ]:
X_train.head()

In [ ]:
  kmeans = KMeans(n_clusters = k, random_state = 42)
  kmeans.fit(drop_string(X_train))
  kmeans.predict(oh)

In [ ]:
  X_train['kmeans'] = kmeans.labels_

  X_train

In [ ]:
print(len(X_train[X_train['kmeans'] == 0]))
len(X_train)

In [32]:
X_train[X_train['kmeans'] == 0]
sample_input

### Kmeans Process Scratch work

In [ ]:
#kmeans.predict(# features for song input)
# Look at cluster predicted
# Extract songs

### OPTIONS FOR NARROWING DOWN SONG CHOICES
## 1. Run clustering recursively
## 2. Compare input to available choices using some tight interval, extract songs from those that remain after filtering.

## Dimension Reduction using T-SNE 

Reduce 13 features to 2 features for better visualization of the data

Visualize using Prof's package

In [ ]:
t1_final.head()

In [ ]:
t1_final.shape

In [ ]:
t1_edit = t1_final[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
t1_edit.head()

In [ ]:
t1_edit.to_numpy()

In [ ]:
# Needs to be an array of vectors

#sorry psalm! this dimension reduction is taking a long ass time

In [ ]:
from sklearn.manifold import TSNE

# Lets dim reduce the ## dimension vectors to 2 dimensions to vizualise the dataset 
# vector_list needs to be a Xarray, shape (n_samples, n_features) or (n_samples, n_samples)

data_embed=TSNE(n_components=2, perplexity=50, verbose=2, method='barnes_hut').fit_transform(t1_edit.to_numpy())


## Parameters
## n_components = number of dimensions you want your data to be reduced
## preplexity =  Number of neighboours to fit the gaussian , normally 30 

In [ ]:
## Vizualise
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
x_axis= data_embed[:,0]
y_axis=data_embed[:,1]

plt.scatter(x_axis, y_axis, s=5)
plt.show() ## The plots vary each time you run them 

## Visualization using Prof's Scatter Plot

In [ ]:
from IPython.display import Javascript
from IPython.display import IFrame

In [ ]:
# clone d3-scatterplot to your local directory
!git clone https://github.com/CAHLR/d3-scatterplot.git

In [ ]:
def show_port(port, data_file, width=600, height=800):
  display(Javascript("""
  (async ()=>{
    fm = document.createElement('iframe')
    fm.src = await google.colab.kernel.proxyPort(%d) + '/index.html?dataset=%s'
    fm.width = '90%%'
    fm.height = '%d'
    fm.frameBorder = 0
    document.body.append(fm)
  })();
  """ % (port, data_file, height)))

In [ ]:
# Format data to be in form
# x | y | feature_1 | feature_2... 
df_scatter = pd.DataFrame({'x': x_axis, 'y': y_axis, 'feature_1':t1_final['Song Name']})
df_scatter.head()

In [ ]:
# Have to convert the df into a tsv file
df_scatter.to_csv('d3-scatterplot/df_scatter.tsv', sep='\t', index=False)


In [ ]:
# Need to figure out how to actually visualize the data in deepnote
# df_scatter.tsv already saved

In [ ]:
port = 8000
data_file = 'df_scatter.tsv'
height = 1600

get_ipython().system_raw('cd d3-scatterplot && python3 -m http.server %d &' % port) 
show_port(port, data_file, height)

In [ ]:
# # run the http server on your local machine
# get_ipython().system_raw('cd d3-scatterplot && python3 -m http.server 8000 &') 

# # To enable, add &[OPTION_NAME]=[FEATURE_NAME] after the dataset=[YOUR_DATASET] in 
# # the URL (e.g. http://localhost:8000/index.html?dataset=example.tsv&color=feature%202 
# # [N.B. %20 is the URL encoding for whitespace characters]).
# # display example.tsv with d3-scatterplot in your jupyter notebook

# IFrame('http://127.0.0.1:8000/index.html?dataset=example.tsv', width=1000, height=1000)